In [20]:
import pandas as pd
import os
import numpy as np

def make_df(data_path):
    df = pd.read_csv(data_path)
    df = df.set_index('...1')

    target_genes = pd.read_csv("/home/sr933/rcc/data/Target_genes.csv")

    
    # Assuming the Target_genes.csv contains a column named 'gene' with row indices
    # Assuming df is your DataFrame and target_indices is the list of target gene names
    target_indices = target_genes['Genes'].values  # Example, adjust this to your actual data

    # Step 1: Identify which target genes are present in df.index
    valid_target_indices = [gene for gene in target_indices if gene in df.index]
    missing_target_indices = [gene for gene in target_indices if gene not in df.index]

    # Step 2: Extract the rows for valid target genes
    filtered_df = df.loc[valid_target_indices]

    # Step 3: Add a row for each missing target gene with zero values (same number of columns as df)
    for missing_gene in missing_target_indices:
        zero_row = pd.Series(0, index=df.columns, name=missing_gene)
        filtered_df = pd.concat([filtered_df, zero_row.to_frame().T])

    # Step 4: Reorder the DataFrame to match the order of target_indices
    filtered_df = filtered_df.loc[target_indices]
    # Step 4: Concatenate the valid target genes and the missing genes (with zero values)

    return filtered_df
main_folder="/data/sr933/scRCC validation/GSE202813_RAW"

# Initialize lists to store data and labels
data = []
labels = []

# Loop through files in the main folder
for file in os.listdir(main_folder):
    if ".gz" in file:
        continue
    print(file)
    data_path = os.path.join(main_folder, file)
    
    # Create a numpy array from the file
    ar = make_df(data_path)  # Ensure `make_nparray` is properly implemented
    data.append(ar.to_numpy().T)
    
    # Step 2: Extract column names from the first CSV
    column_names = ar.columns
    
    # Step 2: Extract the base name of the first file (without extension)


    # Step 3: Construct the path to the second CSV file
    folder_path = "/data/sr933/scRCC validation/GSE202813_RAW"
    second_csv_path = os.path.join(folder_path, file+".gz_cell_annotation.csv")

    # Step 3: Read the second CSV file that contains the column names and cell-type labels
    cell_type_data = pd.read_csv(second_csv_path)

    # Step 4: Create a dictionary to map column names to labels
    # Assuming the first column in the second CSV is the column name and the last column is the label
    label_dict = pd.Series(cell_type_data.iloc[:, -1].values, index=cell_type_data.iloc[:, 0].values).to_dict()

    # Step 5: Create a list of labels corresponding to the columns of ar
    labels.append( [label_dict[column_name] for column_name in column_names])


# Combine all data and labels into arrays
data = np.concatenate(data, axis=0)
labels = np.concatenate(labels)
# Validate shapes
print("Data shape:", data.shape)
print("Labels shape:", labels.shape)

GSM6133738_RCC-BM10-Tumor.count.csv
GSM6133739_RCC-BM10-Involve.count.csv
GSM6133740_RCC-BM10-Noninvolved.count.csv
GSM6133741_RCC-BM3-Tumor.count.csv
GSM6133742_RCC-BM4-Tumor.count.csv
GSM6133743_RCC-BM5-Tumor.count.csv
GSM6133744_RCC-BM7-Tumor.count.csv
GSM6133745_RCC-BM8-Tumor.count.csv
GSM6133746_RCC-BM1-Tumor.count.csv
GSM6133747_RCC-BM1-Involve.count.csv
GSM6133748_RCC-BM1-Noninvolved.count.csv
GSM6133749_RCC-BM2-Tumor.count.csv
GSM6133750_RCC-BM2-Involve.count.csv
GSM6133751_RCC-BM2-Noninvolved.count.csv
GSM6133752_RCC-BM9-Involve.count.csv
GSM6133753_RCC-BM9-Noninvolved.count.csv
GSM6133754_RCC-BM9-Tumor.count.csv
GSM6507003_BMM1-Benign-stroma.count.csv
GSM6507004_BMM2-Benign-stroma.count.csv
GSM6507005_BMM3-Benign-stroma.count.csv
GSM6507006_BMM4-Benign-stroma.count.csv
GSM6507007_BMM5-Benign-stroma.count.csv
GSM6507008_BMM6-Benign-stroma.count.csv
GSM6507009_BMM8-Benign-stroma.count.csv
GSM6507010_BMM9-Benign-stroma.count.csv
GSM6507011_BMM7-Benign-stroma.count.csv
GSM6507012

In [21]:
import pickle
validation_data={"X_test": data, "y_test":labels}
# Path to save the pickle file
pickle_file_path = "/data/sr933/scRCC validation/processed data/scRCC_validation_dataset_metastatic.pkl"  # Update with your desired path

# Save the validation_data dictionary as a pickle file
with open(pickle_file_path, "wb") as pkl_file:
    pickle.dump(validation_data, pkl_file)